# Functions Test

El objetivo de este notebook es crear las funciones para los endpoints que se consumirán en la API. Además, se realizarán pruebas para asegurar su funcionamiento adecuado utilizando los dataset creados específicamente para este fin.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [1]:
import sys
import pandas as pd

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3


## 1 Extracción

En esta sección, extraemos los datos de los archivos `endpoint1`, `endpoint2`, `endpoint3` y `endpoint4` que estan en formato parquet.

### 1.1 Extracción de los datos

Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.

In [2]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'data/processed/{name}.parquet', engine='pyarrow')
    return dataframes

parquet_files = ['endpoint1','endpoint2', 'endpoint3', 'endpoint4' ]
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_endpoint1 = dataframes['endpoint1']
df_endpoint2 = dataframes['endpoint2']
df_endpoint3 = dataframes['endpoint3']
df_endpoint4 = dataframes['endpoint4']

## 2 Endpoints

### 2.1 Endpoint 1

def **developer( *`desarrollador` : str* )**:
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora.
Ejemplo de retorno:

| Año  | Cantidad de Items | Contenido Free  |
|------|-------------------|------------------|
| 2023 | 50                | 27%              |
| 2022 | 45                | 25%              |
| xxxx | xx                | xx%              |

In [21]:
def developer(desarrollador: str):
    df_dev = df_endpoint1[df_endpoint1['developer'] == desarrollador].reset_index()
    df_dev['is_free'] = df_dev['price'] == 0
    result = df_dev.groupby('release_year').agg({'item_id': 'count', 'is_free': 'mean'})
    result.columns = ['Cantidad de Items', 'Contenido Free']
    result['Contenido Free'] = (result['Contenido Free'] * 100).round(2).astype(str) + '%'
    return result.reset_index().to_dict('records')

In [22]:
developer('Facepunch Studios')

[{'release_year': '2006', 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'release_year': '2013', 'Cantidad de Items': 1, 'Contenido Free': '0.0%'}]

### 2.2 Endpoint 2

+ def **userdata( *`User_id` : str* )**:
    Debe devolver `cantidad` de dinero gastado por el usuario, el `porcentaje` de recomendación en base a `reviews.recommend == true` y `cantidad de items`.

Ejemplo de retorno:
```json
{
  "Usuario X" : "us213ndjss09sdf",
  "Dinero gastado": 200,
  "% de recomendación": 0.2,
  "cantidad de items": 5
}
```

In [15]:
def userdata(User_id: str):
    df_user = df_endpoint2[df_endpoint2['user_id'] == User_id].reset_index()
    dinero_gastado = df_user['total_spend'][0]
    cantidad_items = df_user['items_count'][0]
    porcentaje_recomendacion = (df_user[df_user['recommend'] == True]['recommend'].count()/cantidad_items * 100).round(2)


    result = {
        "Usuario" : User_id,
        "Dinero gastado": dinero_gastado,
        "% de recomendación": f"{porcentaje_recomendacion}%",
        "Cantidad de items": cantidad_items
    }

    return result

In [16]:
userdata('Ghoustik')

{'Usuario': 'Ghoustik',
 'Dinero gastado': 28.97,
 '% de recomendación': '25.0%',
 'Cantidad de items': 4}

### 2.3 Endpoint 3


+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno:
```json
{
  "Usuario con más horas jugadas para Género X" : "us213ndjss09sdf",
  "Horas jugadas": [
    {"Año": 2013, "Horas": 203},
    {"Año": 2012, "Horas": 100},
    {"Año": 2011, "Horas": 23}
  ]
}


In [7]:
def UserForGenre(genero: str):
    df_genre = df_endpoint3[genero].reset_index()
    user_max_hours = df_genre.groupby('user_id')[genero].sum().idxmax()
    hours_by_year = df_genre[df_genre['user_id'] == user_max_hours].groupby('release_year')[genero].sum().reset_index()
    hours_by_year.columns = ['Año', 'Horas']
    hours_by_year['Horas'] = hours_by_year['Horas'].astype(int)

    result = {
        f"Usuario con más horas jugadas para Género {genero} " : user_max_hours,
        "Horas jugadas": hours_by_year.to_dict('records')
    }

    return result


In [8]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para Género Indie ': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': '1996', 'Horas': 0},
  {'Año': '1998', 'Horas': 0},
  {'Año': '1999', 'Horas': 0},
  {'Año': '2000', 'Horas': 0},
  {'Año': '2001', 'Horas': 0},
  {'Año': '2002', 'Horas': 0},
  {'Año': '2003', 'Horas': 31},
  {'Año': '2004', 'Horas': 0},
  {'Año': '2005', 'Horas': 36},
  {'Año': '2006', 'Horas': 27},
  {'Año': '2007', 'Horas': 17},
  {'Año': '2008', 'Horas': 22},
  {'Año': '2009', 'Horas': 483},
  {'Año': '2010', 'Horas': 361},
  {'Año': '2011', 'Horas': 1810},
  {'Año': '2012', 'Horas': 2619},
  {'Año': '2013', 'Horas': 3361},
  {'Año': '2014', 'Horas': 6021},
  {'Año': '2015', 'Horas': 13057},
  {'Año': '2016', 'Horas': 14366},
  {'Año': '2017', 'Horas': 564}]}

### 2.4 Endpoint 4

+ def **best_developer_year( *`año` : int* )**:
   Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
  
Ejemplo de retorno:
```json
[
  {"Puesto 1" : X},
  {"Puesto 2" : Y},
  {"Puesto 3" : Z}
]
```

In [9]:
def best_developer_year(year: int):
    df_year = df_endpoint4[(df_endpoint4['posted_year'] == str(year)) & (df_endpoint4['recommend'] == True) & (df_endpoint4['sentiment_analysis'] == 2)]
    top_developers = df_year['developer'].value_counts().nlargest(3).index.tolist()

    result = [{"Puesto 1" : top_developers[0]},
              {"Puesto 2" : top_developers[1]},
              {"Puesto 3" : top_developers[2]}]

    return result


In [10]:
best_developer_year(2015)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Facepunch Studios'},
 {'Puesto 3': 'Smartly Dressed Games'}]

### 2.5 Endpoint 5

+ def **developer_reviews_analysis( *`desarrolladora` : str* )**:
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno:
```
{
  'Valve' : [
     Negative = 182,
     Positive = 278
    ]
}
```


In [11]:
def developer_reviews_analysis(desarrolladora: str):
    df_dev = df_endpoint4[df_endpoint4['developer'] == desarrolladora]
    sentiment_counts = df_dev['sentiment_analysis'].value_counts()
    negative_reviews = sentiment_counts.get(0, 0)
    positive_reviews = sentiment_counts.get(1, 0) + sentiment_counts.get(2, 0)

    result = {
        desarrolladora : [
            {"Negative" : negative_reviews},
            {"Positive" : positive_reviews}
        ]
    }

    return result

In [12]:
developer_reviews_analysis('Ubisoft - San Francisco')

{'Ubisoft - San Francisco': [{'Negative': 6}, {'Positive': 10}]}